In [ ]:
! pip install -qU torch transformers datasets faiss-cpu pandas openpyxl sentence-transformers

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.7.0+cu118 requires torch==2.7.0+cu118, but you have torch 2.9.1 which is incompatible.
torchvision 0.22.0 requires torch==2.7.0, but you have torch 2.9.1 which is incompatible.


In [20]:
import torch
import os
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from datasets import Dataset
from sentence_transformers import CrossEncoder

# ZADANIA 1 i 2

## 1. Wczytanie danych

In [36]:
xlsx_path = "tb_chunks.xlsx"
DOC_ID_COL = "anchor"

df = pd.read_excel(xlsx_path) 
print(df.shape)
print(df.columns)

(2836, 21)
Index(['doc_id', 'file_name', 'chunk_id', 'chunk_kind', 'kind', 'section_path',
       'annex_id', 'chapter_id', 'article_id', 'text', 'page_start',
       'page_end', 'reg_no', 'supplement', 'effective_from', 'jurisdiction',
       'tokens', 'level', 'parent_path', 'anchor', 'part_index'],
      dtype='object')


In [4]:
df.head()

,doc_id,file_name,chunk_id,chunk_kind,kind,section_path,annex_id,chapter_id,article_id,text,...,page_end,reg_no,supplement,effective_from,jurisdiction,tokens,level,parent_path,anchor,part_index
0,ECE 160R02-00 EDR cons,ECE 160R02-00 EDR cons.pdf,leaf:ECE 160R02-00 EDR cons:2.49.,LEAF,SECTION,2.49.,NaN,NaN,NaN,"2.49. ""Stability control"" means any device tha...",...,5,UN R160,02 series of amendments,NaT,UNECE,27,2.0,2.,UN R160 §2.49. (eff. n/a),NaN
1,ECE 160R02-00 EDR cons,ECE 160R02-00 EDR cons.pdf,leaf:ECE 160R02-00 EDR cons:3.2.3.,LEAF,SECTION,3.2.3.,NaN,NaN,NaN,3.2.3. Instructions for retrieving data from t...,...,6,UN R160,02 series of amendments,NaT,UNECE,13,3.0,3.2.,UN R160 §3.2.3. (eff. n/a),NaN
2,ECE 161R00-04 Unauthorized use and device agai...,ECE 161R00-04 Unauthorized use and device agai...,leaf:ECE 161R00-04 Unauthorized use and device...,LEAF,SECTION,4.5.,NaN,NaN,NaN,"4.5. \nIf a type conforms to a type approved, ...",...,5,UN R161,Supplement 4 to the original version of Regula...,2024-06-15,UNECE,117,2.0,4.,UN R161 §4.5. (eff. 2024-06-15),NaN
3,ECE 162R00-05 Immobilizers cons,ECE 162R00-05 Immobilizers cons.pdf,leaf:ECE 162R00-05 Immobilizers cons:5.1.9.,LEAF,SECTION,5.1.9.,NaN,NaN,NaN,5.1.9. \nIt shall not be possible for an immob...,...,7,UN R162,Supplement 5 to the original version of the Re...,2024-06-15,UNECE,157,3.0,5.1.,UN R162 §5.1.9. (eff. 2024-06-15),NaN
4,ECE 160R02-00 EDR cons,ECE 160R02-00 EDR cons.pdf,leaf:ECE 160R02-00 EDR cons:2.56.,LEAF,SECTION,2.56.,NaN,NaN,NaN,"2.56. ""Time to deploy, side air bag/curtain"" m...",...,5,UN R160,02 series of amendments,NaT,UNECE,50,2.0,2.,UN R160 §2.56. (eff. n/a),NaN


In [5]:
TEXT_COL = "text"

In [6]:
# proste czyszczenie: wyrzuć wiersze bez tekstu
df = df[df[TEXT_COL].notna()].reset_index(drop=True)
print("Po czyszczeniu:", df.shape)

Po czyszczeniu: (2836, 21)


## 2. Model embeddingowy

Tokenizer przygotowuje wektor wejściowy. Model go rozumie i produkuje embedding.

In [ ]:
model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_

In [ ]:
def cls_pooling(model_output):
    # [batch, seq_len, hidden] -> [batch, hidden], bierzemy token [CLS]
    return model_output.last_hidden_state[:, 0]

Funkcja get_embeddings tokenizuje teksty, przepuszcza je przez model MPNet, wyciąga embedding tokena CLS jako reprezentację zdania i zwraca go w formie numpy float32. W ten sposób otrzymujemy wektory, które można porównywać przez dot-product i używać w FAISS lub w pipeline retrieve & rerank.
{
    'input_ids': tensor([[...], [...]]),
    'attention_mask': tensor([[1,1,1,0,...], ...])
}


In [65]:
@torch.no_grad()
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=512,  # na wszelki wypadek
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    embeddings = cls_pooling(model_output)
    #embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
    return embeddings.cpu().numpy()

In [12]:
test_emb = get_embeddings([df[TEXT_COL].iloc[0]])
print(test_emb.shape)  # (1, 768)

(1, 768)


In [13]:
test_emb

array([[-4.72055413e-02, -6.56372130e-01, -2.76724339e-01,
        -3.46932471e-01,  1.47241414e-01, -6.83599710e-02,
         2.71481782e-01,  2.72080973e-02,  9.88795161e-02,
         1.87647432e-01, -2.17194837e-02, -6.02835044e-03,
        -1.71283521e-02,  8.16687346e-02,  1.22140385e-01,
        -2.02151269e-01,  2.59146720e-01,  1.14801109e-01,
         1.64699793e-01,  1.29657045e-01, -1.98402062e-01,
        -1.04573071e-01, -2.84784138e-01, -4.42993641e-02,
        -2.24792033e-01,  1.39552370e-01,  4.33117032e-01,
        -1.29539400e-01,  4.94786911e-03, -2.81166911e-01,
         2.92516440e-01,  3.40127975e-01,  1.46495134e-01,
         1.25227302e-01, -9.76357696e-05, -4.22990844e-02,
         9.17379111e-02, -1.92948237e-01, -1.01870392e-02,
         3.40387821e-01, -8.20228010e-02, -3.25414985e-02,
        -1.65557519e-01,  5.67735061e-02, -4.21527773e-02,
         3.02372910e-02, -1.10934019e-01,  1.67308122e-01,
         5.87345734e-02,  1.42638966e-01,  3.10694873e-0

## 3. Zamiana na HuggingFace Dataset + policzenie embeddings

In [14]:
# pandas -> HF Dataset
ds = Dataset.from_pandas(df, preserve_index=False)

In [ ]:
def embed_batch(batch):
    embs = get_embeddings(batch[TEXT_COL])          # (batch, hidden)
    embs = embs.astype("float32")                 
    return {"embeddings": embs}

embeddings_ds = ds.map(
    embed_batch,
    batched=True,
    batch_size=32,
)

Map:   0%|          | 0/2836 [00:00<?, ? examples/s]

In [ ]:
embeddings_ds.save_to_disk("regs_chunks_with_embeddings")

Do wczytania embeddingów i indeksu nie od zera

In [ ]:
#embeddings_ds = Dataset.load_from_disk("regs_chunks_with_embeddings")
#embeddings_ds.load_faiss_index("embeddings", "faiss_index.bin")

## 4. FAISS index

Ponieważ zbiór danych zawiera około 2800 embeddingów (znacznie poniżej progu 100k), zgodnie z zaleceniami FAISS (‘Guidelines to choose an index’) najlepszym rozwiązaniem jest indeks typu IndexFlatIP, który zapewnia dokładne wyszukiwanie bez konieczności uczenia struktury i bez strat jakości. Zastosowanie go dało optymalny kompromis między prostotą implementacji, szybkością oraz jakością wyników.

In [16]:
embeddings_ds.add_faiss_index(column="embeddings")
embeddings_ds.save_faiss_index("embeddings", "faiss_index.bin")

  0%|          | 0/3 [00:00<?, ?it/s]

In [40]:
print(type(embeddings_ds[0]["embeddings"]))
print(np.array(embeddings_ds[0]["embeddings"]).shape)

<class 'list'>
(768,)


In [41]:
print("Liczba wierszy:", embeddings_ds.num_rows)

Liczba wierszy: 2836


In [42]:
def get_dir_size(path="."):
    total = 0
    for root, dirs, files in os.walk(path):
        for f in files:
            fp = os.path.join(root, f)
            total += os.path.getsize(fp)
    return total

path = "regs_chunks_with_embeddings"
size_bytes = get_dir_size(path)
size_mb = size_bytes / (1024 * 1024)

print(f"Rozmiar datasetu HF: {size_mb:.2f} MB")

Rozmiar datasetu HF: 11.31 MB


In [43]:
size_bytes = os.path.getsize("faiss_index.bin")
print(f"Rozmiar FAISS indexu: {size_bytes / (1024*1024):.2f} MB")

Rozmiar FAISS indexu: 8.31 MB


### Semantic search

- wyszukiwanie wektorowe (Nearest Neighbors),
- użycie indeksu FAISS (jeśli został załadowany),
- sortowanie wyników według podobieństwa,
- zwrócenie top-k najbardziej podobnych przykładów.

In [44]:
def embed_query(query: str) -> np.ndarray:
    q_emb = get_embeddings([query])  # shape (1, 768)
    return q_emb.astype("float32")

def semantic_search(query: str, k: int = 5):
    q_emb = embed_query(query)
    scores, samples = embeddings_ds.get_nearest_examples(
        "embeddings",
        q_emb,
        k=k,
    )
    # zamiana samples -> dataframe, żeby było wygodnie
    res_df = pd.DataFrame.from_dict(samples)
    res_df["score"] = scores
    res_df.sort_values("score", ascending=False, inplace=True)
    return res_df

## 2a i 2b (kilka wyszukiwań + analiza błędów)

In [51]:
def pretty_search(query: str, k: int = 5):
    print(f"\n=== QUERY: {query} ===\n")
    results = semantic_search(query, k=k)
    for i, (_, row) in enumerate(results.iterrows(), start=1):
        print(f"#{i}  SCORE: {row['score']:.3f}")
        print(f"ANCHOR: {row.get('anchor')}")
        print(f"KIND: {row.get('chunk_kind')}, REG: {row.get('reg_no')}, SECTION: {row.get('section_path')}")
        print(f"TEXT:\n{row[TEXT_COL][:3000]}...\n")
        print("-" * 80)

In [52]:
pretty_search("requirements for trailer direction indicator failure tell-tale", k=5)


=== QUERY: requirements for trailer direction indicator failure tell-tale ===

#1  SCORE: 29.622
ANCHOR: UN R048 §6.5.1. (eff. 2024-09-22)
KIND: LEAF, REG: UN R048, SECTION: 6.5.1.
TEXT:
6.5.1. 
Presence (see figure below) 
 
Mandatory. Types of direction-indicator lamps fall into categories (1, 1a, 1b, 
2a, 2b, 5 and 6) the assembly of which on one vehicle constitutes an 
arrangement ("A" and "B").  
 
Arrangement "A" shall apply to all motor vehicles.  
 
Arrangement "B" shall apply to trailers only....

--------------------------------------------------------------------------------
#2  SCORE: 29.493
ANCHOR: UN R048 §6.5.8. (eff. 2024-09-22)
KIND: LEAF, REG: UN R048, SECTION: 6.5.8.
TEXT:
6.5.8. 
Tell-tale 
Operating 
tell-tale 
mandatory 
for 
direction-indicator 
lamps 
of 
categories 1, 1a, 1b, 2a and 2b. It may be visual or auditory or both. If it is 
visual it shall be a flashing light which, at least in the event of the 
malfunction of any of these direction-indicator lamps, 

In [53]:
pretty_search("what is the minimum deceleration for service braking", k=5)


=== QUERY: what is the minimum deceleration for service braking ===

#1  SCORE: 30.073
ANCHOR: UN R13-H §2.9.3. (eff. 2023-04-01)
KIND: LEAF, REG: UN R13-H, SECTION: 2.9.3.
TEXT:
2.9.3. the braking force can be easily regulated with sufficient precision....

--------------------------------------------------------------------------------
#2  SCORE: 29.904
ANCHOR: UN R13-H Annex 3 - part 6 (eff. 2023-04-01)
KIND: ANNEX_PART, REG: UN R13-H, SECTION: None
TEXT:
case where the failure condition causes the electric component to deliver its maximum braking force. 
2.3. Parking braking system
2.3.1. The parking braking system shall be capable of holding the laden vehicle stationary on a 20 per cent up or down
gradient.
2.3.2. On vehicles to which the coupling of a trailer is authorized, the parking braking system of the motor vehicle shall
be capable of holding the combination of vehicles stationary on a 12 per cent up or down gradient.
2.3.3. If the control device is manual, the force appli

In [54]:
pretty_search("cybersecurity management system requirements", k=5)


=== QUERY: cybersecurity management system requirements ===

#1  SCORE: 35.411
ANCHOR: UN R155 §7.2.2.5. (eff. n/a)
KIND: LEAF, REG: UN R155, SECTION: 7.2.2.5.
TEXT:
7.2.2.5. The vehicle manufacturer shall be required to demonstrate how their Cyber Security Management
System will manage dependencies that may exist with contracted suppliers, service providers or manufacturer’s
sub-organizations in regards of the requirements of paragraph 7.2.2.2....

--------------------------------------------------------------------------------
#2  SCORE: 35.354
ANCHOR: UN R155 Annex 5 - part 8 (eff. n/a)
KIND: ANNEX_PART, REG: UN R155, SECTION: None
TEXT:
rue if software has not been
tested to verify that known bad code/bugs is
not present and reduce the risk of unknown
bad code/bugs being present
M23
Cybersecurity best practices for software and
hardware development shall be followed.
Cybersecurity testing with adequate coverage
28.2
Using remainders from development (e.g.
debug ports, JTAG ports, 

In [55]:
pretty_search("regulation about automatic headlamp levelling", k=5)


=== QUERY: regulation about automatic headlamp levelling ===

#1  SCORE: 28.562
ANCHOR: UN R121 §40. (eff. 2023-09-24)
KIND: LEAF, REG: UN R121, SECTION: 40.
TEXT:
40.
Headlamp levelling
13/
Control
No...

--------------------------------------------------------------------------------
#2  SCORE: 27.691
ANCHOR: UN R121 §40. (eff. 2023-09-24)
KIND: PACK, REG: UN R121, SECTION: 40.
TEXT:
40.
40.
Headlamp levelling
13/
Control
No...

--------------------------------------------------------------------------------
#3  SCORE: 22.418
ANCHOR: UN R048 §2.2.1.3. (eff. 2024-09-22)
KIND: LEAF, REG: UN R048, SECTION: 2.2.1.3.
TEXT:
2.2.1.3. 
The headlamp-levelling system;...

--------------------------------------------------------------------------------
#4  SCORE: 17.510
ANCHOR: UN R048 §6.2.6.2.1. (eff. 2024-09-22)
KIND: LEAF, REG: UN R048, SECTION: 6.2.6.2.1.
TEXT:
6.2.6.2.1. 
In the case where a headlamp levelling device is necessary to satisfy the 
requirements of paragraphs 6.2.6.1.1. and 

## ZADANIE 3 porównanie semantic search bez indeksu z FAISS

Chcemy mieć dwie wersje:

- bez FAISS – bruteforce: liczymy podobieństwo query do wszystkich embeddingów,

- z FAISS (HF).

In [56]:
# --- Przygotowanie pod ewaluację (bez FAISS) ---
# Wszystkie embeddingi jako pojedyncza macierz (N, dim)
all_embs = np.vstack(embeddings_ds["embeddings"]).astype("float32")
doc_ids_all = np.array(embeddings_ds[DOC_ID_COL])  # czyli 'anchor'

In [ ]:
# ---- Semantic search BEZ FAISS (brute-force na all_embs) ----
def semantic_search_no_index(query: str, k: int = 5):
    q_emb = embed_query(query)           # (1, dim)
    q = q_emb[0]                         # (dim,)

    scores = all_embs @ q                # (N,)

    topk_idx = np.argsort(-scores)[:k]
    topk_scores = scores[topk_idx]
    topk_ids = doc_ids_all[topk_idx]     

    res_df = df.iloc[topk_idx].copy()
    res_df["score"] = topk_scores
    res_df["doc_id_eval"] = topk_ids    
    return res_df

# ---- Semantic search Z FAISS (wersja z HF indexem) ----
def semantic_search_faiss(query: str, k: int = 5):
    q_emb = embed_query(query)
    scores, samples = embeddings_ds.get_nearest_examples(
        "embeddings",
        q_emb,
        k=k,
    )
    res_df = pd.DataFrame.from_dict(samples)
    res_df["score"] = scores

    if DOC_ID_COL in res_df.columns:          
        res_df["doc_id_eval"] = res_df[DOC_ID_COL]
    else:
        # awaryjnie
        res_df["doc_id_eval"] = res_df.index.astype(str)

    res_df.sort_values("score", ascending=False, inplace=True)
    return res_df

In [58]:
q_test = "requirements for trailer direction indicator failure tell-tale"

print("=== BEZ FAISS ===")
print(semantic_search_no_index(q_test, k=5)[["doc_id_eval", "score"]])

print("\n=== Z FAISS (HF) ===")
print(semantic_search_faiss(q_test, k=5)[["doc_id_eval", "score"]])


=== BEZ FAISS ===
                                 doc_id_eval      score
2509       UN R048 §6.5.8. (eff. 2024-09-22)  31.273653
2575       UN R048 §6.6.9. (eff. 2024-09-22)  31.081383
671       UN R79 §5.3.1.7. (eff. 2024-01-05)  30.688171
2341       UN R048 §6.5.1. (eff. 2024-09-22)  30.625032
836   UN R79 §5.6.4.5.5.1. (eff. 2024-01-05)  29.945217

=== Z FAISS (HF) ===
                              doc_id_eval      score
4       UN R048 §6.5.1. (eff. 2024-09-22)  29.621685
3       UN R048 §6.5.8. (eff. 2024-09-22)  29.493113
2  UN R79 §5.6.4.5.5.1. (eff. 2024-01-05)  27.791832
1       UN R048 §6.6.9. (eff. 2024-09-22)  26.043217
0      UN R79 §5.3.1.7. (eff. 2024-01-05)  24.076689


In [60]:
import time

def benchmark_search(query: str, k: int = 5, repeats: int = 5):
    """
    Mierzy średni czas wyszukiwania:
    - brute-force (no index)
    - FAISS (HF)
    """
    # --- BEZ FAISS ---
    t0 = time.perf_counter()
    for _ in range(repeats):
        res_no_index = semantic_search_no_index(query, k=k)
    t_no_index = (time.perf_counter() - t0) / repeats

    # --- Z FAISS ---
    t0 = time.perf_counter()
    for _ in range(repeats):
        res_faiss = semantic_search_faiss(query, k=k)
    t_faiss = (time.perf_counter() - t0) / repeats

    print(f"\n=== WYNIKI WYSZUKIWANIA DLA QUERY ===")
    print(query)
    print("\n--- BEZ FAISS (brute-force) ---")
    print(res_no_index[["doc_id_eval", "score"]])
    print(f"Czas średni: {t_no_index*1000:.3f} ms")

    print("\n--- Z FAISS (HF Index) ---")
    print(res_faiss[["doc_id_eval", "score"]])
    print(f"Czas średni: {t_faiss*1000:.3f} ms")

    print("\n--- PORÓWNANIE ---")
    print(f"FAISS jest około {t_no_index / t_faiss:.1f}× szybszy\n")


In [61]:
q_test = "requirements for trailer direction indicator failure tell-tale"
benchmark_search(q_test, k=5, repeats=10)



=== WYNIKI WYSZUKIWANIA DLA QUERY ===
requirements for trailer direction indicator failure tell-tale

--- BEZ FAISS (brute-force) ---
                                 doc_id_eval     score
2509       UN R048 §6.5.8. (eff. 2024-09-22)  4.686687
2575       UN R048 §6.6.9. (eff. 2024-09-22)  4.657874
671       UN R79 §5.3.1.7. (eff. 2024-01-05)  4.598947
2341       UN R048 §6.5.1. (eff. 2024-09-22)  4.589485
836   UN R79 §5.6.4.5.5.1. (eff. 2024-01-05)  4.487608
Czas średni: 51.294 ms

--- Z FAISS (HF Index) ---
                            doc_id_eval      score
4  UN R048 §6.10.4.3. (eff. 2024-09-22)  27.821386
3      UN R79 §9.1.2. (eff. 2024-01-05)  27.766724
2  UN R048 §6.15.4.3. (eff. 2024-09-22)  27.620623
1  UN R048 §6.18.4.1. (eff. 2024-09-22)  27.573639
0   UN R048 §6.1.4.2. (eff. 2024-09-22)  26.400051
Czas średni: 22.733 ms

--- PORÓWNANIE ---
FAISS jest około 2.3× szybszy



## ZADANIE 4 Cross - encoder

In [9]:
# Cross-encoder do re-rankingu (re-ranker)
CROSS_ENCODER_NAME = "cross-encoder/ms-marco-MiniLM-L-6-v2"

cross_encoder = CrossEncoder(CROSS_ENCODER_NAME, device=device)
cross_encoder.model.eval()


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

d:\users\sf34875\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\users\sf34875\.cache\huggingface\hub\models--cross-encoder--ms-marco-MiniLM-L-6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not ins

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-1

In [ ]:
def retrieve_contexts_local(query: str, top_k: int = 5) -> list[dict]:
    """
    Bi-encoder + FAISS (HF index) - zwraca listę kontekstów jako słowniki.
    """
    q_emb = embed_query(query)
    scores, samples = embeddings_ds.get_nearest_examples(
        "embeddings",
        q_emb,
        k=top_k,
    )
    rows = pd.DataFrame.from_dict(samples)
    rows["score_bi"] = scores

    contexts = []
    for _, row in rows.iterrows():
        contexts.append({
            "text": str(row[TEXT_COL]),
            "anchor": row.get("anchor"),
            "doc_id": row.get("doc_id"),
            "reg_no": row.get("reg_no"),
            "section_path": row.get("section_path"),
            "annex_id": row.get("annex_id"),
            "score_bi": float(row["score_bi"]),
        })
    return contexts

In [ ]:
def retrieve_contexts_local_reranked(query: str,
                                     top_k_bi: int = 30,
                                     top_k: int = 5) -> list[dict]:
    """
    1) Bi-encoder + FAISS (HF): bierzemy top_k_bi kandydatów
    2) Cross-encoder: liczymy dokładne podobieństwa dla (query, text)
    3) Sortujemy po score cross-encodera i zwracamy top_k
    """
    # 1) wstępne wyszukiwanie bi-enkoderem
    q_emb = embed_query(query)
    scores_bi, samples = embeddings_ds.get_nearest_examples(
        "embeddings",
        q_emb,
        k=top_k_bi,
    )
    rows = pd.DataFrame.from_dict(samples)
    rows["score_bi"] = scores_bi

    # 2) przygotowanie par (query, passage) dla cross-encodera
    pairs = [(query, str(t)) for t in rows[TEXT_COL].tolist()]

    with torch.no_grad():
        scores_ce = cross_encoder.predict(pairs)  # numpy array (top_k_bi,)

    rows["score_ce"] = scores_ce

    # 3) sortujemy wg cross-encodera i bierzemy top_k
    rows_sorted = rows.sort_values("score_ce", ascending=False).head(top_k)

    contexts = []
    for _, row in rows_sorted.iterrows():
        contexts.append({
            "text": str(row[TEXT_COL]),
            "anchor": row.get("anchor"),
            "doc_id": row.get("doc_id"),
            "reg_no": row.get("reg_no"),
            "section_path": row.get("section_path"),
            "annex_id": row.get("annex_id"),
            "score_bi": float(row["score_bi"]),
            "score_ce": float(row["score_ce"]),
        })
    return contexts

In [62]:
def debug_search_with_and_without_rerank(query: str, k: int = 5):
    print(f"\n=== QUERY ===\n{query}\n")

    print(">>> Bi-encoder + FAISS (bez re-rankera):")
    base_ctx = retrieve_contexts_local(query, top_k=k)
    for i, c in enumerate(base_ctx, 1):
        snippet = c["text"][:2000].replace("\n", " ")
        print(f"{i}. [BI] {c.get('anchor')}  |  score_bi={c['score_bi']:.3f}")
        print(f"    snippet: {snippet}...")

    print("\n>>> Bi-encoder + FAISS + Cross-encoder re-rank (top_k_bi=30 -> top_k=5):")
    reranked_ctx = retrieve_contexts_local_reranked(query, top_k_bi=30, top_k=k)
    for i, c in enumerate(reranked_ctx, 1):
        snippet = c["text"][:2000].replace("\n", " ")
        print(f"{i}. [CE] {c.get('anchor')}  |  score_bi={c['score_bi']:.3f}, score_ce={c['score_ce']:.3f}")
        print(f"    snippet: {snippet}...")


In [66]:
debug_search_with_and_without_rerank("requirements for trailer direction indicator failure tell-tale", k=5)


=== QUERY ===
requirements for trailer direction indicator failure tell-tale

>>> Bi-encoder + FAISS (bez re-rankera):
1. [BI] UN R79 §5.3.1.7. (eff. 2024-01-05)  |  score_bi=24.077
    snippet: 5.3.1.7. In the case of trailers the requirements of paragraphs 5.2.2. and 6.3.4.1. shall also be met when there is a failure in the steering system....
2. [BI] UN R048 §6.6.9. (eff. 2024-09-22)  |  score_bi=26.043
    snippet: 6.6.9.  Other requirements  As specified in paragraph 6.5.9. If a power-driven vehicle is equipped to  draw a trailer the hazard warning signal control shall also be capable of  bringing the direction-indicator lamps on the trailer into action. The hazard  warning signal shall be able to function even if the device which starts or  stops the propulsion system is in a position which makes it impossible to start  the propulsion system....
3. [BI] UN R79 §5.6.4.5.5.1. (eff. 2024-01-05)  |  score_bi=27.792
    snippet: 5.6.4.5.5.1. In case the vehicle is coupled to a traile

In [68]:
debug_search_with_and_without_rerank("what is the minimum deceleration for service braking", k=5)


=== QUERY ===
what is the minimum deceleration for service braking

>>> Bi-encoder + FAISS (bez re-rankera):
1. [BI] UN R13-H Annex 3 - part 3 (eff. 2023-04-01)  |  score_bi=27.300
    snippet:  no specific recommendation; (c) At a maximum level resulting from automatic charge control on the vehicle; or (d) When the tests are conducted without a regenerative braking component regardless of the state of charge of the batteries. 1.4.1.2.4. the limits prescribed for minimum performance, both for tests with the vehicle unladen and for tests with the vehicle laden, shall be those laid down hereunder; the vehicle shall satisfy both the prescribed stopping distance and the prescribed mean fully developed deceleration, but it may not be necessary to actually measure both parameters; 1.4.1.2.5. the road shall be level: unless otherwise specified each test may comprise up to six stops including any needed for familiarization 1.4.2. Type-0 test with engine disconnected, service braking in accord

## Metryki ewaluacyjne

In [ ]:
EVAL_QUERIES = [
    {
        "query": "requirements for trailer direction indicator failure tell-tale",
        "relevant_doc_ids": [
            "UN_R48_6.5.8.1_a",
            "UN_R48_6.5.8.2_b",
        ],
    },
    {
        "query": "what is the minimum deceleration for service braking",
        "relevant_doc_ids": [
            "UN_R13_5.2.2.2",
        ],
    },
    {
        "query": "cybersecurity management system requirements",
        "relevant_doc_ids": [
            "UN_R155_5.1",
        ],
    },
    {
        "query": "regulation about automatic headlamp levelling",
        "relevant_doc_ids": [
            "UN_R48_6.2.6.2",
        ],
    },
]


In [ ]:
# ---- 7. Metryki: Hit@k, Recall@k, MRR@k ----
def metrics_for_single_query(ranked_doc_ids, relevant_doc_ids, k: int):
    """
    ranked_doc_ids: lista doc_id_eval w kolejności od najlepszego
    relevant_doc_ids: lista doc_id_eval uznanych za poprawne
    """
    ranked_doc_ids = list(ranked_doc_ids)[:k]
    relevant = set(relevant_doc_ids)

    # Hit@k
    hit = 1.0 if any(doc_id in relevant for doc_id in ranked_doc_ids) else 0.0

    # Recall@k
    if len(relevant) == 0:
        recall = 0.0
    else:
        retrieved_relevant = sum(1 for doc_id in ranked_doc_ids if doc_id in relevant)
        recall = retrieved_relevant / len(relevant)

    # MRR@k
    rr = 0.0
    for rank, doc_id in enumerate(ranked_doc_ids, start=1):
        if doc_id in relevant:
            rr = 1.0 / rank
            break

    return {"hit": hit, "recall": recall, "mrr": rr}


def evaluate_backend(backend_fn, k: int = 5):
    """
    backend_fn: funkcja przyjmująca (query, k) i zwracająca DataFrame z kolumną 'doc_id_eval'
                np. semantic_search_no_index lub semantic_search_faiss
    """
    rows = []

    for item in EVAL_QUERIES:
        query = item["query"]
        rel_ids = item["relevant_doc_ids"]

        res_df = backend_fn(query, k=k)
        ranked_ids = res_df["doc_id_eval"].tolist()

        m = metrics_for_single_query(ranked_ids, rel_ids, k=k)
        rows.append({"query": query, **m})

    metrics_df = pd.DataFrame(rows)
    summary = {
        "hit@k_mean": metrics_df["hit"].mean(),
        "recall@k_mean": metrics_df["recall"].mean(),
        "mrr@k_mean": metrics_df["mrr"].mean(),
    }
    return metrics_df, summary


In [ ]:
def backend_bi(query: str, k: int = 5):
    ctxs = retrieve_contexts_local(query, top_k=k)
    res_df = pd.DataFrame(ctxs)
    # doc_id_eval = anchor
    res_df["doc_id_eval"] = res_df[DOC_ID_COL]   # czyli 'anchor'
    return res_df


def backend_bi_ce(query: str, k: int = 5):
    ctxs = retrieve_contexts_local_reranked(query, top_k_bi=30, top_k=k)
    res_df = pd.DataFrame(ctxs)
    res_df["doc_id_eval"] = res_df[DOC_ID_COL]   # czyli 'anchor'
    return res_df
